In [1]:
import numpy as np
import tensorflow as tf


print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.3.2


In [2]:
dataset = np.load("dataset_overlap.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")

In [3]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [4]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [5]:
def split_dataset_and_normalize(perc=0.8):
    shuffled_dataset, shuffled_labels = unison_shuffled_copies(dataset,classes)
    split_point = int(len(dataset)*perc)
    train_ds = shuffled_dataset[:split_point]
    train_labels = shuffled_labels[:split_point]
    val_ds = shuffled_dataset[split_point:]
    val_labels = shuffled_labels[split_point:]
    std_dev = np.std(train_ds)
    mean = np.mean(train_ds)
    maximum = np.max(train_ds)
    min = np.min(train_ds)
    scale = max(maximum, abs(min))
    train_ds = normalize(train_ds, std_dev, mean, scale)
    val_ds = normalize(val_ds, std_dev, mean, scale)
    return train_ds, train_labels, val_ds, val_labels

train_ds, train_labels, val_ds, val_labels = split_dataset_and_normalize()

In [39]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Bidirectional(LSTM(192, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.3)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection (None, 384)               25665024  
_________________________________________________________________
dropout_28 (Dropout)         (None, 384)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               49280     
_________________________________________________________________
dropout_29 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_30 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 32)               

In [41]:
filepath = "audio_model_bidirectional_LSTM_overlap.h5"
cb1 = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
cb2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

cbs = [cb1,cb2]

In [42]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

In [43]:
EPOCHS = 80
history = model.fit(
    x= train_ds,
    y = train_labels,
    validation_data=(val_ds, val_labels),
    batch_size=16,
    epochs=EPOCHS,
    callbacks=cbs
)

Epoch 1/80
38/38 [==============================] - 9s 231ms/step - loss: 0.7306 - accuracy: 0.4587 - val_loss: 0.7284 - val_accuracy: 0.4430
Epoch 2/80
38/38 [==============================] - 8s 216ms/step - loss: 0.7267 - accuracy: 0.5076 - val_loss: 0.7271 - val_accuracy: 0.4228
Epoch 3/80
38/38 [==============================] - 8s 217ms/step - loss: 0.7259 - accuracy: 0.4705 - val_loss: 0.7257 - val_accuracy: 0.4295
Epoch 4/80
38/38 [==============================] - 8s 215ms/step - loss: 0.7240 - accuracy: 0.4874 - val_loss: 0.7242 - val_accuracy: 0.4832
Epoch 5/80
38/38 [==============================] - 8s 213ms/step - loss: 0.7216 - accuracy: 0.5177 - val_loss: 0.7225 - val_accuracy: 0.4765
Epoch 6/80
38/38 [==============================] - 8s 214ms/step - loss: 0.7182 - accuracy: 0.5245 - val_loss: 0.7209 - val_accuracy: 0.4966
Epoch 7/80
38/38 [==============================] - 8s 214ms/step - loss: 0.7175 - accuracy: 0.5261 - val_loss: 0.7190 - val_accuracy: 0.5034
Epoch 

38/38 [==============================] - 8s 220ms/step - loss: 0.4339 - accuracy: 0.9174 - val_loss: 0.5297 - val_accuracy: 0.8054
Epoch 59/80
38/38 [==============================] - 8s 220ms/step - loss: 0.4293 - accuracy: 0.9207 - val_loss: 0.5270 - val_accuracy: 0.8054
Epoch 60/80
38/38 [==============================] - 8s 217ms/step - loss: 0.4222 - accuracy: 0.9376 - val_loss: 0.5244 - val_accuracy: 0.8188
Epoch 61/80
38/38 [==============================] - 8s 220ms/step - loss: 0.4223 - accuracy: 0.9444 - val_loss: 0.5224 - val_accuracy: 0.7919
Epoch 62/80
38/38 [==============================] - 8s 216ms/step - loss: 0.4163 - accuracy: 0.9325 - val_loss: 0.5200 - val_accuracy: 0.8121
Epoch 63/80
38/38 [==============================] - 8s 215ms/step - loss: 0.4137 - accuracy: 0.9309 - val_loss: 0.5179 - val_accuracy: 0.8188
Epoch 64/80
38/38 [==============================] - 8s 218ms/step - loss: 0.4105 - accuracy: 0.9562 - val_loss: 0.5163 - val_accuracy: 0.8054
Epoch 65/80

In [38]:
del model

NameError: name 'model' is not defined

In [12]:
model2 = keras.models.load_model("audio_model_bidirectional_LSTM_overlap.h5")

model2.evaluate(val_ds)